In [ ]:
%sql -- Extração de ordens (AUFK) 

WITH ordens_filtradas AS (
    SELECT aufk.DI_SEQUENCE_NUMBER, aufk.AUFNR, aufk.MANDT
    FROM poseidon_uc.saps4hanarawview.aufk AS aufk
    WHERE aufk.AUART IN ('GIA','GAA','GRA') 
    AND aufk.BUKRS IN ('GHIN', 'GHSM', 'EPEI')
    -- AND aufk.BUKRS IN ('GHIN')
    AND aufk.DI_SEQUENCE_NUMBER = (
        SELECT MAX(a.DI_SEQUENCE_NUMBER)
        FROM poseidon_uc.saps4hanarawview.aufk AS a
        WHERE a.AUFNR = aufk.AUFNR
    )
    AND aufk.DI_OPERATION_TYPE != 'D'
    AND ( 
        -- 610000048871, 610000053688, 610000053689, 610000053730, 610000053892, 610000054003
        aufk.AUFNR = 610000053688 OR
        aufk.AUFNR = 610000053689 OR
        aufk.AUFNR = 610000053730
    )
    LIMIT 10  -- Processar em lotes
)
SELECT
    aufk.*
    -- aufk.AUFNR AS ordem,
    -- aufk.AUART AS tipo_ordem,
    -- aufk.BUKRS AS empresa,
    -- aufk.KTEXT AS texto_breve,
    -- aufk.ERDAT AS data_inicio_real,
    -- aufk.WERKS AS centro_trabalho,
    -- aufk.VAPLZ AS centro_trabalho_princ

FROM ordens_filtradas of
INNER JOIN poseidon_uc.saps4hanarawview.aufk AS aufk
    ON aufk.AUFNR = of.AUFNR 
    AND aufk.DI_SEQUENCE_NUMBER = of.DI_SEQUENCE_NUMBER
    AND aufk.MANDT = of.MANDT

ORDER BY aufk.AUFNR

In [ ]:
%sql -- Extração enriquecida de ordens (AUFK) com a tabela AFIH

WITH ordens_filtradas AS (
    SELECT aufk.DI_SEQUENCE_NUMBER, aufk.AUFNR, aufk.MANDT
    FROM poseidon_uc.saps4hanarawview.aufk AS aufk
    WHERE aufk.AUART IN ('GIA','GAA','GRA') 
    AND aufk.BUKRS IN ('GHIN', 'GHSM', 'EPEI')
    AND aufk.DI_SEQUENCE_NUMBER = (
        SELECT MAX(a.DI_SEQUENCE_NUMBER)
        FROM poseidon_uc.saps4hanarawview.aufk AS a
        WHERE a.AUFNR = aufk.AUFNR
    )
    AND aufk.DI_OPERATION_TYPE != 'D'
    -- AND ( 
    --     -- 610000048871, 610000053688, 610000053689, 610000053730, 610000053892, 610000054003
    --     aufk.AUFNR = 610000053688 OR
    --     aufk.AUFNR = 610000053689 OR
    --     aufk.AUFNR = 610000053730
    -- )
    LIMIT 10  -- Processar em lotes
),
afih_filtrado AS (
    -- SELECT afih.DI_SEQUENCE_NUMBER, afih.MANDT, afih.AUFNR, afih.PRIOK, afih.ARTPR
    SELECT 
        afih.*,
        t356_t.PRIOKX AS texto_prioridade

    FROM poseidon_uc.saps4hanarawview.afih AS afih

    LEFT JOIN poseidon_uc.saps4hanarawview.t356_t AS t356_t
        ON t356_t.PRIOK = afih.PRIOK
        AND t356_t.ARTPR = afih.ARTPR
        AND t356_t.SPRAS = 'P'
    
    WHERE afih.DI_SEQUENCE_NUMBER = (
        SELECT MAX(a.DI_SEQUENCE_NUMBER)
        FROM poseidon_uc.saps4hanarawview.afih AS a
        WHERE a.MANDT = afih.MANDT 
        AND a.AUFNR = afih.AUFNR
    )
)
SELECT
    aufk.AUFNR AS ordem,
    -- aufk.AUART AS tipo_ordem,
    -- aufk.BUKRS AS empresa,
    -- aufk.KTEXT AS texto_breve,
    -- aufk.ERDAT AS data_criacao,
    -- aufk.WERKS AS centro_trabalho,
    
    afih.*,

    afih.texto_prioridade AS texto_prioridade

FROM ordens_filtradas AS aufk

-- LEFT JOIN poseidon_uc.saps4hanarawview.afko AS afko
--     ON afko.MANDT = aufk.MANDT AND afko.AUFNR = aufk.AUFNR

LEFT JOIN afih_filtrado AS afih
    ON afih.MANDT = aufk.MANDT AND afih.AUFNR = aufk.AUFNR

ORDER BY aufk.AUFNR

In [ ]:
%sql -- Extração enriquecida de ordens (AUFK) com a tabela AFIH

WITH ordens_filtradas AS (
    SELECT aufk.DI_SEQUENCE_NUMBER, aufk.AUFNR, aufk.MANDT
    FROM poseidon_uc.saps4hanarawview.aufk AS aufk
    WHERE aufk.AUART IN ('GIA','GAA','GRA') 
    AND aufk.BUKRS IN ('GHIN', 'GHSM', 'EPEI')
    AND aufk.DI_SEQUENCE_NUMBER = (
        SELECT MAX(a.DI_SEQUENCE_NUMBER)
        FROM poseidon_uc.saps4hanarawview.aufk AS a
        WHERE a.AUFNR = aufk.AUFNR
    )
    AND aufk.DI_OPERATION_TYPE != 'D'
    -- AND ( 
    --     -- 610000048871, 610000053688, 610000053689, 610000053730, 610000053892, 610000054003
    --     aufk.AUFNR = 610000053688 OR
    --     aufk.AUFNR = 610000053689 OR
    --     aufk.AUFNR = 610000053730
    -- )
    LIMIT 10  -- Processar em lotes
),
afih_filtrado AS (
    -- SELECT afih.DI_SEQUENCE_NUMBER, afih.MANDT, afih.AUFNR, afih.PRIOK, afih.ARTPR
    SELECT 
        afih.*

    FROM poseidon_uc.saps4hanarawview.afih AS afih
    
    WHERE afih.DI_SEQUENCE_NUMBER = (
        SELECT MAX(a.DI_SEQUENCE_NUMBER)
        FROM poseidon_uc.saps4hanarawview.afih AS a
        WHERE a.MANDT = afih.MANDT 
        AND a.AUFNR = afih.AUFNR
    )
)
SELECT
    aufk.AUFNR AS ordem,
    -- aufk.AUART AS tipo_ordem,
    -- aufk.BUKRS AS empresa,
    -- aufk.KTEXT AS texto_breve,
    -- aufk.ERDAT AS data_criacao,
    -- aufk.WERKS AS centro_trabalho,
    
    afih.*,
    aufk.AUFNR AS left_afih_right_t356_t,
    t356_t.*

FROM ordens_filtradas AS aufk

-- LEFT JOIN poseidon_uc.saps4hanarawview.afko AS afko
--     ON afko.MANDT = aufk.MANDT AND afko.AUFNR = aufk.AUFNR

LEFT JOIN afih_filtrado AS afih
    ON afih.MANDT = aufk.MANDT AND afih.AUFNR = aufk.AUFNR

LEFT JOIN poseidon_uc.saps4hanarawview.t356_t AS t356_t
    ON t356_t.PRIOK = afih.PRIOK
    AND t356_t.ARTPR = afih.ARTPR
    AND t356_t.SPRAS = 'P'

ORDER BY aufk.AUFNR

In [ ]:
%sql -- Extração enriquecida de ordens (AUFK) com a tabela AFKO

WITH ordens_filtradas AS (
    SELECT aufk.DI_SEQUENCE_NUMBER, aufk.AUFNR, aufk.MANDT
    FROM poseidon_uc.saps4hanarawview.aufk AS aufk
    WHERE aufk.AUART IN ('GIA','GAA','GRA') 
    AND aufk.BUKRS IN ('GHIN', 'GHSM', 'EPEI')
    -- AND aufk.BUKRS IN ('GHIN')
    AND aufk.DI_SEQUENCE_NUMBER = (
        SELECT MAX(a.DI_SEQUENCE_NUMBER)
        FROM poseidon_uc.saps4hanarawview.aufk AS a
        WHERE a.AUFNR = aufk.AUFNR
    )
    AND aufk.DI_OPERATION_TYPE != 'D'
    LIMIT 10  -- Processar em lotes
),
afko_filtrado AS (
    -- SELECT afih.DI_SEQUENCE_NUMBER, afih.MANDT, afih.AUFNR, afih.PRIOK, afih.ARTPR
    SELECT afko.*
    FROM poseidon_uc.saps4hanarawview.afko AS afko
    WHERE afko.DI_SEQUENCE_NUMBER = (
        SELECT MAX(a.DI_SEQUENCE_NUMBER)
        FROM poseidon_uc.saps4hanarawview.afko AS a
        WHERE a.MANDT = afko.MANDT 
        AND a.AUFNR = afko.AUFNR
    )
)
SELECT
    aufk.AUFNR AS ordem,
    -- aufk.AUART AS tipo_ordem,
    -- aufk.BUKRS AS empresa,
    -- aufk.KTEXT AS texto_breve,
    -- aufk.ERDAT AS data_criacao,
    -- aufk.WERKS AS centro_trabalho,
    
    afko.*

FROM ordens_filtradas of
INNER JOIN poseidon_uc.saps4hanarawview.aufk AS aufk
    ON aufk.AUFNR = of.AUFNR 
    AND aufk.DI_SEQUENCE_NUMBER = of.DI_SEQUENCE_NUMBER
    AND aufk.MANDT = of.MANDT

LEFT JOIN afko_filtrado AS afko
    ON afko.MANDT = aufk.MANDT AND afko.AUFNR = aufk.AUFNR

-- WHERE

ORDER BY aufk.AUFNR
-- LIMIT 10

In [ ]:
%sql -- Extração enriquecida de ordens (AUFK) com a tabela AFRU

WITH ordens_filtradas AS (
    SELECT aufk.DI_SEQUENCE_NUMBER, aufk.AUFNR, aufk.MANDT
    FROM poseidon_uc.saps4hanarawview.aufk AS aufk
    WHERE aufk.AUART IN ('GIA','GAA','GRA') 
    AND aufk.BUKRS IN ('GHIN', 'GHSM', 'EPEI')
    -- AND aufk.BUKRS IN ('GHIN')
    AND aufk.DI_SEQUENCE_NUMBER = (
        SELECT MAX(a.DI_SEQUENCE_NUMBER)
        FROM poseidon_uc.saps4hanarawview.aufk AS a
        WHERE a.AUFNR = aufk.AUFNR
    )
    AND aufk.DI_OPERATION_TYPE != 'D'
    LIMIT 10  -- Processar em lotes
),
afru_filtrado AS (
    -- SELECT afih.DI_SEQUENCE_NUMBER, afih.MANDT, afih.AUFNR, afih.PRIOK, afih.ARTPR
    SELECT afru.*
    FROM poseidon_uc.saps4hanarawview.afru AS afru
    WHERE afru.DI_SEQUENCE_NUMBER = (
        SELECT MAX(a.DI_SEQUENCE_NUMBER)
        FROM poseidon_uc.saps4hanarawview.afru AS a
        WHERE a.MANDT = afru.MANDT 
        AND a.AUFNR = afru.AUFNR
    )
)
SELECT
    aufk.AUFNR AS ordem,
    -- aufk.AUART AS tipo_ordem,
    -- aufk.BUKRS AS empresa,
    -- aufk.KTEXT AS texto_breve,
    -- aufk.ERDAT AS data_criacao,
    -- aufk.WERKS AS centro_trabalho,
    
    afru.*

FROM ordens_filtradas of
INNER JOIN poseidon_uc.saps4hanarawview.aufk AS aufk
    ON aufk.AUFNR = of.AUFNR 
    AND aufk.DI_SEQUENCE_NUMBER = of.DI_SEQUENCE_NUMBER
    AND aufk.MANDT = of.MANDT

LEFT JOIN afru_filtrado AS afru
    ON afru.MANDT = aufk.MANDT AND afru.AUFNR = aufk.AUFNR

-- WHERE

ORDER BY aufk.AUFNR
-- LIMIT 10

In [ ]:
%sql -- Extração enriquecida de ordens (AUFK) com a tabela AFVC
-- afvc completa (sem filtro) pra ordens específicas

WITH ordens_filtradas AS (
    SELECT aufk.DI_SEQUENCE_NUMBER, aufk.AUFNR, aufk.MANDT
    FROM poseidon_uc.saps4hanarawview.aufk AS aufk
    WHERE aufk.AUART IN ('GIA','GAA','GRA') 
    AND aufk.BUKRS IN ('GHIN', 'GHSM', 'EPEI')
    -- AND aufk.BUKRS IN ('GHIN')
    AND aufk.DI_SEQUENCE_NUMBER = (
        SELECT MAX(a.DI_SEQUENCE_NUMBER)
        FROM poseidon_uc.saps4hanarawview.aufk AS a
        WHERE a.AUFNR = aufk.AUFNR
    )
    AND aufk.DI_OPERATION_TYPE != 'D'
    AND ( -- retorno nulo para essas ordens, tentar alternativa com a tabela afru
        aufk.AUFNR = 610000042263 OR
        aufk.AUFNR = 610000042264 OR
        aufk.AUFNR = 610000051699 OR
        aufk.AUFNR = 610000051729 OR
        aufk.AUFNR = 610000051755 OR
        aufk.AUFNR = 610000051772 OR
        aufk.AUFNR = 610000052429 OR
        aufk.AUFNR = 610000052431
    ) 
    -- LIMIT 10  -- Processar em lotes
),
afko_filtrado AS (
    -- SELECT afih.DI_SEQUENCE_NUMBER, afih.MANDT, afih.AUFNR, afih.PRIOK, afih.ARTPR
    SELECT afko.*
    FROM poseidon_uc.saps4hanarawview.afko AS afko
    WHERE afko.DI_SEQUENCE_NUMBER = (
        SELECT MAX(a.DI_SEQUENCE_NUMBER)
        FROM poseidon_uc.saps4hanarawview.afko AS a
        WHERE a.MANDT = afko.MANDT 
        AND a.AUFNR = afko.AUFNR
    )
)
-- ,
-- afvc_filtrado AS (
--     SELECT afvc.*
--     FROM poseidon_uc.saps4hanarawview.afvc AS afvc
--     WHERE afvc.VORNR = '0010'
--     AND afvc.DI_SEQUENCE_NUMBER = (
--         SELECT MAX(a.DI_SEQUENCE_NUMBER)
--         FROM poseidon_uc.saps4hanarawview.afvc AS a
--         WHERE a.MANDT = afvc.MANDT
--         AND a.AUFPL = afvc.AUFPL
--         AND a.VORNR = '0010'
--     )
-- )
SELECT
    aufk.AUFNR AS ordem,
    -- aufk.AUART AS tipo_ordem,
    -- aufk.BUKRS AS empresa,
    -- aufk.KTEXT AS texto_breve,
    -- aufk.ERDAT AS data_criacao,
    -- aufk.WERKS AS centro_trabalho,
    
    afvc.*

FROM ordens_filtradas of
INNER JOIN poseidon_uc.saps4hanarawview.aufk AS aufk
    ON aufk.AUFNR = of.AUFNR 
    AND aufk.DI_SEQUENCE_NUMBER = of.DI_SEQUENCE_NUMBER
    AND aufk.MANDT = of.MANDT

LEFT JOIN afko_filtrado AS afko
    ON afko.MANDT = aufk.MANDT AND afko.AUFNR = aufk.AUFNR

LEFT JOIN poseidon_uc.saps4hanarawview.afvc AS afvc
    ON afvc.MANDT = afko.MANDT AND afvc.AUFPL = afko.AUFPL

-- WHERE

ORDER BY aufk.AUFNR
-- LIMIT 10

In [ ]:
%sql -- Extração enriquecida de ordens (AUFK) com a tabela AFVC


WITH ordens_filtradas AS (
    SELECT aufk.DI_SEQUENCE_NUMBER, aufk.AUFNR, aufk.MANDT
    FROM poseidon_uc.saps4hanarawview.aufk AS aufk
    WHERE aufk.AUART IN ('GIA','GAA','GRA') 
    AND aufk.BUKRS IN ('GHIN', 'GHSM', 'EPEI')
    -- AND aufk.BUKRS IN ('GHIN')
    AND aufk.DI_SEQUENCE_NUMBER = (
        SELECT MAX(a.DI_SEQUENCE_NUMBER)
        FROM poseidon_uc.saps4hanarawview.aufk AS a
        WHERE a.AUFNR = aufk.AUFNR
    )
    AND aufk.DI_OPERATION_TYPE != 'D'
    LIMIT 10  -- Processar em lotes
),
afko_filtrado AS (
    -- SELECT afih.DI_SEQUENCE_NUMBER, afih.MANDT, afih.AUFNR, afih.PRIOK, afih.ARTPR
    SELECT afko.*
    FROM poseidon_uc.saps4hanarawview.afko AS afko
    WHERE afko.DI_SEQUENCE_NUMBER = (
        SELECT MAX(a.DI_SEQUENCE_NUMBER)
        FROM poseidon_uc.saps4hanarawview.afko AS a
        WHERE a.MANDT = afko.MANDT 
        AND a.AUFNR = afko.AUFNR
    )
),
afvc_filtrado AS (
    SELECT afvc.*
    FROM poseidon_uc.saps4hanarawview.afvc AS afvc
    WHERE afvc.VORNR IN ('0010', '010', '10')
    AND afvc.DI_SEQUENCE_NUMBER = (
        SELECT MAX(a.DI_SEQUENCE_NUMBER)
        FROM poseidon_uc.saps4hanarawview.afvc AS a
        WHERE a.MANDT = afvc.MANDT
        AND a.AUFPL = afvc.AUFPL
        AND a.VORNR IN ('0010', '010', '10')
    )
)
SELECT
    aufk.AUFNR AS ordem,
    -- aufk.AUART AS tipo_ordem,
    -- aufk.BUKRS AS empresa,
    -- aufk.KTEXT AS texto_breve,
    -- aufk.ERDAT AS data_criacao,
    -- aufk.WERKS AS centro_trabalho,
    
    afvc.*

FROM ordens_filtradas of
INNER JOIN poseidon_uc.saps4hanarawview.aufk AS aufk
    ON aufk.AUFNR = of.AUFNR 
    AND aufk.DI_SEQUENCE_NUMBER = of.DI_SEQUENCE_NUMBER
    AND aufk.MANDT = of.MANDT

LEFT JOIN afko_filtrado AS afko
    ON afko.MANDT = aufk.MANDT AND afko.AUFNR = aufk.AUFNR

LEFT JOIN afvc_filtrado AS afvc
    ON afvc.MANDT = afko.MANDT AND afvc.AUFPL = afko.AUFPL

-- WHERE afvc.DI_SEQUENCE_NUMBER IS NULL

ORDER BY aufk.AUFNR
-- LIMIT 10

In [ ]:
%sql -- Extração enriquecida de ordens (AUFK) com a tabela ILOA (diretamente)
-- Retorno nulo, nao funciona

WITH ordens_filtradas AS (
    SELECT aufk.DI_SEQUENCE_NUMBER, aufk.AUFNR, aufk.MANDT
    FROM poseidon_uc.saps4hanarawview.aufk AS aufk
    WHERE aufk.AUART IN ('GIA','GAA','GRA') 
    AND aufk.BUKRS IN ('GHIN', 'GHSM', 'EPEI')
    -- AND aufk.BUKRS IN ('GHIN')
    AND aufk.DI_SEQUENCE_NUMBER = (
        SELECT MAX(a.DI_SEQUENCE_NUMBER)
        FROM poseidon_uc.saps4hanarawview.aufk AS a
        WHERE a.AUFNR = aufk.AUFNR
    )
    AND aufk.DI_OPERATION_TYPE != 'D'
    LIMIT 10  -- Processar em lotes
),
iloa_filtrado AS (
    -- SELECT afih.DI_SEQUENCE_NUMBER, afih.MANDT, afih.AUFNR, afih.PRIOK, afih.ARTPR
    SELECT iloa.*
    FROM poseidon_uc.saps4hanarawview.iloa AS iloa
    WHERE iloa.DI_SEQUENCE_NUMBER = (
        SELECT MAX(a.DI_SEQUENCE_NUMBER)
        FROM poseidon_uc.saps4hanarawview.iloa AS a
        WHERE a.MANDT = iloa.MANDT 
        AND a.AUFNR = iloa.AUFNR
    )
)
SELECT
    aufk.AUFNR AS ordem,
    -- aufk.AUART AS tipo_ordem,
    -- aufk.BUKRS AS empresa,
    -- aufk.KTEXT AS texto_breve,
    -- aufk.ERDAT AS data_criacao,
    -- aufk.WERKS AS centro_trabalho,
    
    iloa.*

FROM ordens_filtradas of
INNER JOIN poseidon_uc.saps4hanarawview.aufk AS aufk
    ON aufk.AUFNR = of.AUFNR 
    AND aufk.DI_SEQUENCE_NUMBER = of.DI_SEQUENCE_NUMBER
    AND aufk.MANDT = of.MANDT

LEFT JOIN iloa_filtrado AS iloa
    ON iloa.MANDT = aufk.MANDT AND iloa.AUFNR = aufk.AUFNR

ORDER BY aufk.AUFNR

In [ ]:
%sql -- Extração enriquecida de ordens (AUFK) com a tabela ILOA (culuna afih.iloan como intermediaria)

WITH ordens_filtradas AS (
    SELECT aufk.DI_SEQUENCE_NUMBER, aufk.AUFNR, aufk.MANDT
    FROM poseidon_uc.saps4hanarawview.aufk AS aufk
    WHERE aufk.AUART IN ('GIA','GAA','GRA') 
    AND aufk.BUKRS IN ('GHIN', 'GHSM', 'EPEI')
    -- AND aufk.BUKRS IN ('GHIN')
    AND aufk.DI_SEQUENCE_NUMBER = (
        SELECT MAX(a.DI_SEQUENCE_NUMBER)
        FROM poseidon_uc.saps4hanarawview.aufk AS a
        WHERE a.AUFNR = aufk.AUFNR
    )
    AND aufk.DI_OPERATION_TYPE != 'D'
    AND ( 
        -- 610000048871, 610000053688, 610000053689, 610000053730, 610000053892, 610000054003
        -- aufk.AUFNR = 610000048871 OR
        -- aufk.AUFNR = 610000053892 OR
        -- aufk.AUFNR = 610000054003 OR
        aufk.AUFNR = 610000053688 OR
        aufk.AUFNR = 610000053689 OR
        aufk.AUFNR = 610000053730
    )
    LIMIT 10  -- Processar em lotes
),
afih_filtrado AS (
    -- SELECT afih.DI_SEQUENCE_NUMBER, afih.MANDT, afih.AUFNR, afih.PRIOK, afih.ARTPR
    SELECT afih.*
    FROM poseidon_uc.saps4hanarawview.afih AS afih
    WHERE afih.DI_SEQUENCE_NUMBER = (
        SELECT MAX(a.DI_SEQUENCE_NUMBER)
        FROM poseidon_uc.saps4hanarawview.afih AS a
        WHERE a.MANDT = afih.MANDT 
        AND a.AUFNR = afih.AUFNR
    )
),
iloa_filtrado AS (
    -- SELECT afih.DI_SEQUENCE_NUMBER, afih.MANDT, afih.AUFNR, afih.PRIOK, afih.ARTPR
    SELECT iloa.*
    FROM poseidon_uc.saps4hanarawview.iloa AS iloa
    WHERE iloa.DI_SEQUENCE_NUMBER = (
        SELECT MAX(a.DI_SEQUENCE_NUMBER)
        FROM poseidon_uc.saps4hanarawview.iloa AS a
        WHERE a.MANDT = iloa.MANDT 
        AND a.ILOAN = iloa.ILOAN
    )
)
SELECT
    aufk.AUFNR AS ordem,
    aufk.DI_SEQUENCE_NUMBER AS aufk_di_sequence_number,
    -- aufk.AUART AS tipo_ordem,
    -- aufk.BUKRS AS empresa,
    -- aufk.KTEXT AS texto_breve,
    -- aufk.ERDAT AS data_criacao,
    -- aufk.WERKS AS centro_trabalho,
    afih.ILOAN AS afih_iloan,
    afih.DI_SEQUENCE_NUMBER AS afih_di_sequence_number,
    iloa.*

FROM ordens_filtradas of
INNER JOIN poseidon_uc.saps4hanarawview.aufk AS aufk
    ON aufk.AUFNR = of.AUFNR 
    AND aufk.DI_SEQUENCE_NUMBER = of.DI_SEQUENCE_NUMBER
    AND aufk.MANDT = of.MANDT

-- LEFT JOIN poseidon_uc.saps4hanarawview.afko AS afko
--     ON afko.MANDT = aufk.MANDT AND afko.AUFNR = aufk.AUFNR

LEFT JOIN afih_filtrado AS afih
    ON afih.MANDT = aufk.MANDT AND afih.AUFNR = aufk.AUFNR

LEFT JOIN iloa_filtrado AS iloa
    ON iloa.MANDT = afih.MANDT AND iloa.ILOAN = afih.ILOAN

-- WHERE

ORDER BY aufk.AUFNR

In [ ]:
%sql -- Extração enriquecida de ordens (AUFK) com a tabela JEST (culuna OBJNR como intermediaria)

WITH ordens_filtradas AS (
    SELECT aufk.DI_SEQUENCE_NUMBER, aufk.AUFNR, aufk.MANDT, aufk.OBJNR
    FROM poseidon_uc.saps4hanarawview.aufk AS aufk
    WHERE aufk.AUART IN ('GIA','GAA','GRA') 
    AND aufk.BUKRS IN ('GHIN', 'GHSM', 'EPEI')
    -- AND aufk.BUKRS IN ('GHIN')
    AND aufk.DI_SEQUENCE_NUMBER = (
        SELECT MAX(a.DI_SEQUENCE_NUMBER)
        FROM poseidon_uc.saps4hanarawview.aufk AS a
        WHERE a.AUFNR = aufk.AUFNR
    )
    AND aufk.DI_OPERATION_TYPE != 'D'
    LIMIT 100  -- Processar em lotes
),
jest_filtrado AS (
    SELECT 
        jest.MANDT,
        jest.OBJNR,
        jest.STAT,
        jest.INACT,
        jest.DI_OPERATION_TYPE,
        jest.DI_SEQUENCE_NUMBER

    FROM poseidon_uc.saps4hanarawview.jest AS jest
    WHERE jest.DI_SEQUENCE_NUMBER = (
        SELECT MAX(a.DI_SEQUENCE_NUMBER)
        FROM poseidon_uc.saps4hanarawview.jest AS a
        WHERE a.MANDT = jest.MANDT
        AND a.OBJNR = jest.OBJNR
        AND a.STAT = jest.STAT
    )
    AND jest.DI_OPERATION_TYPE != 'D'
),
jsto_filtrado AS (
    SELECT 
        jsto.OBJNR,
        jsto.STSMA,
        jsto.DI_OPERATION_TYPE,
        jsto.DI_SEQUENCE_NUMBER

    FROM poseidon_uc.saps4hanarawview.jsto AS jsto
    WHERE jsto.DI_SEQUENCE_NUMBER = (
        SELECT MAX(a.DI_SEQUENCE_NUMBER)
        FROM poseidon_uc.saps4hanarawview.jsto AS a
        WHERE a.OBJNR = jsto.OBJNR
    )
    AND jsto.DI_OPERATION_TYPE != 'D'
),
tj30_filtrado AS (
    SELECT 
        tj30.*

    FROM poseidon_uc.saps4hanarawview.tj30 AS tj30
    WHERE tj30.DI_SEQUENCE_NUMBER = (
        SELECT MAX(a.DI_SEQUENCE_NUMBER)
        FROM poseidon_uc.saps4hanarawview.tj30 AS a
        WHERE a.ESTAT = tj30.ESTAT
        AND a.STSMA = tj30.STSMA
    )
    AND tj30.DI_OPERATION_TYPE != 'D'
),
tj30t_filtrado AS (
    SELECT 
        tj30t.ESTAT,
        tj30t.SPRAS,
        tj30t.STSMA,
        tj30t.TXT04,
        tj30t.DI_OPERATION_TYPE,
        tj30t.DI_SEQUENCE_NUMBER

    FROM poseidon_uc.saps4hanarawview.tj30t AS tj30t
    WHERE tj30t.SPRAS = 'P'
    AND tj30t.DI_SEQUENCE_NUMBER = (
        SELECT MAX(a.DI_SEQUENCE_NUMBER)
        FROM poseidon_uc.saps4hanarawview.tj30t AS a
        WHERE a.ESTAT = tj30t.ESTAT
        AND a.SPRAS = 'P'
        AND a.STSMA = tj30t.STSMA
    )
    AND tj30t.DI_OPERATION_TYPE != 'D'
),
status_usuario AS (
    SELECT 
        aufk.AUFNR AS ordem,
        tj30.LINEP,
        tj30t.TXT04

    FROM ordens_filtradas AS aufk

    LEFT JOIN jest_filtrado AS jest
        ON jest.MANDT = aufk.MANDT 
        AND jest.OBJNR = aufk.OBJNR
        AND jest.STAT LIKE 'E%'
        AND jest.INACT IS NULL

    LEFT JOIN jsto_filtrado AS jsto
        ON jsto.OBJNR = aufk.OBJNR

    LEFT JOIN tj30_filtrado AS tj30
        ON jsto.STSMA = tj30.STSMA AND jest.STAT = tj30.ESTAT

    LEFT JOIN tj30t_filtrado AS tj30t
        ON jest.STAT = tj30t.ESTAT
        AND tj30t.SPRAS = 'P'
        AND tj30t.STSMA = jsto.STSMA
)
SELECT
    stat.ordem,
    -- aufk.AUART AS tipo_ordem,
    -- aufk.BUKRS AS empresa,
    -- aufk.KTEXT AS texto_breve,
    -- aufk.ERDAT AS data_criacao,
    -- aufk.WERKS AS centro_trabalho,
    -- STRING_AGG(tj30t.TXT04, ' ') WITHIN GROUP (ORDER BY tj30t.TXT04) AS status_usuario
    -- CONCAT_WS(' ', COLLECT_LIST(stat.TXT04)) AS status_usuario
    CONCAT_WS(' ', 
        transform(
            array_sort(collect_list(struct(stat.LINEP, stat.TXT04))),
            x -> x.TXT04
        )
    ) AS status_usuario

FROM status_usuario AS stat 

GROUP BY stat.ordem


In [ ]:
%sql -- Extração enriquecida de ordens (AUFK) com a tabela JEST (culuna OBJNR como intermediaria)
-- sem concatenação


WITH ordens_filtradas AS (
    -- SELECT aufk.DI_SEQUENCE_NUMBER, aufk.AUFNR, aufk.MANDT
    SELECT aufk.*
    FROM poseidon_uc.saps4hanarawview.aufk AS aufk
    WHERE aufk.AUART IN ('GIA','GAA','GRA') 
    AND aufk.BUKRS IN ('GHIN', 'GHSM', 'EPEI')
    -- AND aufk.BUKRS IN ('GHIN')
    AND aufk.DI_SEQUENCE_NUMBER = (
        SELECT MAX(a.DI_SEQUENCE_NUMBER)
        FROM poseidon_uc.saps4hanarawview.aufk AS a
        WHERE a.AUFNR = aufk.AUFNR
    )
    AND aufk.DI_OPERATION_TYPE != 'D'
    -- AND ( 
        
    --     aufk.AUFNR = 610000007039 OR
    --     aufk.AUFNR = 610000018920 OR
    --     aufk.AUFNR = 610000025056 OR
    --     aufk.AUFNR = 610000031153 OR
    --     aufk.AUFNR = 610000041209 OR
    --     aufk.AUFNR = 610000036041 OR
    --     aufk.AUFNR = 610000048610
    -- )
    LIMIT 100  -- Processar em lotes
),
jest_filtrado AS (
    SELECT 
        jest.MANDT,
        jest.OBJNR,
        jest.STAT,
        jest.INACT,
        jest.DI_OPERATION_TYPE,
        jest.DI_SEQUENCE_NUMBER

    FROM poseidon_uc.saps4hanarawview.jest AS jest
    WHERE jest.DI_SEQUENCE_NUMBER = (
        SELECT MAX(a.DI_SEQUENCE_NUMBER)
        FROM poseidon_uc.saps4hanarawview.jest AS a
        WHERE a.MANDT = jest.MANDT
        AND a.OBJNR = jest.OBJNR
        AND a.STAT = jest.STAT
    )
    AND jest.DI_OPERATION_TYPE != 'D'
),
jsto_filtrado AS (
    SELECT 
        jsto.OBJNR,
        jsto.STSMA,
        jsto.DI_OPERATION_TYPE,
        jsto.DI_SEQUENCE_NUMBER

    FROM poseidon_uc.saps4hanarawview.jsto AS jsto
    WHERE jsto.DI_SEQUENCE_NUMBER = (
        SELECT MAX(a.DI_SEQUENCE_NUMBER)
        FROM poseidon_uc.saps4hanarawview.jsto AS a
        WHERE a.OBJNR = jsto.OBJNR
    )
    AND jsto.DI_OPERATION_TYPE != 'D'
),
tj30_filtrado AS (
    SELECT 
        tj30.*

    FROM poseidon_uc.saps4hanarawview.tj30 AS tj30
    WHERE tj30.DI_SEQUENCE_NUMBER = (
        SELECT MAX(a.DI_SEQUENCE_NUMBER)
        FROM poseidon_uc.saps4hanarawview.tj30 AS a
        WHERE a.ESTAT = tj30.ESTAT
        AND a.STSMA = tj30.STSMA
    )
    AND tj30.DI_OPERATION_TYPE != 'D'
),
tj30t_filtrado AS (
    SELECT 
        tj30t.ESTAT,
        tj30t.SPRAS,
        tj30t.STSMA,
        tj30t.TXT04,
        tj30t.DI_OPERATION_TYPE,
        tj30t.DI_SEQUENCE_NUMBER

    FROM poseidon_uc.saps4hanarawview.tj30t AS tj30t
    WHERE tj30t.SPRAS = 'P'
    AND tj30t.DI_SEQUENCE_NUMBER = (
        SELECT MAX(a.DI_SEQUENCE_NUMBER)
        FROM poseidon_uc.saps4hanarawview.tj30t AS a
        WHERE a.ESTAT = tj30t.ESTAT
        AND a.SPRAS = 'P'
        AND a.STSMA = tj30t.STSMA
    )
    AND tj30t.DI_OPERATION_TYPE != 'D'
)
SELECT
    aufk.AUFNR AS ordem,
    -- aufk.AUART AS tipo_ordem,
    -- aufk.BUKRS AS empresa,
    -- aufk.KTEXT AS texto_breve,
    -- aufk.ERDAT AS data_criacao,
    -- aufk.WERKS AS centro_trabalho,
    -- STRING_AGG(tj30t.TXT04, ' ') WITHIN GROUP (ORDER BY tj30t.TXT04) AS status_usuario
    -- CONCAT_WS(' ', COLLECT_LIST(tj30t.TXT04)) AS status_usuario
    jest.*,
    aufk.AUFNR as left_JEST_right_JSTO,
    jsto.*,
    aufk.AUFNR as left_JSTO_right_tj30,
    tj30.*,
    aufk.AUFNR as left_tj30_right_tj30t,
    tj30t.*

FROM ordens_filtradas AS aufk

LEFT JOIN jest_filtrado AS jest
    ON jest.MANDT = aufk.MANDT 
    AND jest.OBJNR = aufk.OBJNR
    AND jest.STAT LIKE 'E%'
    AND jest.INACT IS NULL

LEFT JOIN jsto_filtrado AS jsto
    ON jsto.OBJNR = aufk.OBJNR

LEFT JOIN tj30_filtrado AS tj30
    ON jsto.STSMA = tj30.STSMA AND jest.STAT = tj30.ESTAT

LEFT JOIN tj30t_filtrado AS tj30t
    ON jest.STAT = tj30t.ESTAT
    AND tj30t.SPRAS = 'P'
    AND tj30t.STSMA = jsto.STSMA

-- GROUP BY aufk.AUFNR


In [ ]:
%sql -- Extração enriquecida de ordens (AUFK) com a tabela JEST (culuna OBJNR como intermediaria)
-- sem concatenação
-- sem jest


WITH ordens_filtradas AS (
    -- SELECT aufk.DI_SEQUENCE_NUMBER, aufk.AUFNR, aufk.MANDT
    SELECT aufk.*
    FROM poseidon_uc.saps4hanarawview.aufk AS aufk
    WHERE aufk.AUART IN ('GIA','GAA','GRA') 
    AND aufk.BUKRS IN ('GHIN', 'GHSM', 'EPEI')
    -- AND aufk.BUKRS IN ('GHIN')
    AND aufk.DI_SEQUENCE_NUMBER = (
        SELECT MAX(a.DI_SEQUENCE_NUMBER)
        FROM poseidon_uc.saps4hanarawview.aufk AS a
        WHERE a.AUFNR = aufk.AUFNR
    )
    AND aufk.DI_OPERATION_TYPE != 'D'
    -- AND ( 
        
    --     aufk.AUFNR = 610000007039 OR
    --     aufk.AUFNR = 610000018920 OR
    --     aufk.AUFNR = 610000025056 OR
    --     aufk.AUFNR = 610000031153 OR
    --     aufk.AUFNR = 610000041209 OR
    --     aufk.AUFNR = 610000036041 OR
    --     aufk.AUFNR = 610000048610
    -- )
    LIMIT 10  -- Processar em lotes
),
jcds_filtrado AS (
    SELECT 
        jcds.*

    FROM poseidon_uc.saps4hanarawview.jcds AS jcds
    WHERE jcds.DI_SEQUENCE_NUMBER = (
        SELECT MAX(a.DI_SEQUENCE_NUMBER)
        FROM poseidon_uc.saps4hanarawview.jcds AS a
        WHERE a.MANDT = jcds.MANDT
        AND a.OBJNR = jcds.OBJNR
        AND a.STAT = jcds.STAT
    )
    AND jcds.DI_OPERATION_TYPE != 'D'
),
jsto_filtrado AS (
    SELECT 
        jsto.OBJNR,
        jsto.STSMA,
        jsto.DI_OPERATION_TYPE,
        jsto.DI_SEQUENCE_NUMBER

    FROM poseidon_uc.saps4hanarawview.jsto AS jsto
    WHERE jsto.DI_SEQUENCE_NUMBER = (
        SELECT MAX(a.DI_SEQUENCE_NUMBER)
        FROM poseidon_uc.saps4hanarawview.jsto AS a
        WHERE a.OBJNR = jsto.OBJNR
    )
    AND jsto.DI_OPERATION_TYPE != 'D'
),
tj30_filtrado AS (
    SELECT 
        tj30.*

    FROM poseidon_uc.saps4hanarawview.tj30 AS tj30
    WHERE tj30.DI_SEQUENCE_NUMBER = (
        SELECT MAX(a.DI_SEQUENCE_NUMBER)
        FROM poseidon_uc.saps4hanarawview.tj30 AS a
        WHERE a.ESTAT = tj30.ESTAT
        AND a.STSMA = tj30.STSMA
    )
    AND tj30.DI_OPERATION_TYPE != 'D'
),
tj30t_filtrado AS (
    SELECT 
        tj30t.ESTAT,
        tj30t.SPRAS,
        tj30t.STSMA,
        tj30t.TXT04,
        tj30t.DI_OPERATION_TYPE,
        tj30t.DI_SEQUENCE_NUMBER

    FROM poseidon_uc.saps4hanarawview.tj30t AS tj30t
    WHERE tj30t.SPRAS = 'P'
    AND tj30t.DI_SEQUENCE_NUMBER = (
        SELECT MAX(a.DI_SEQUENCE_NUMBER)
        FROM poseidon_uc.saps4hanarawview.tj30t AS a
        WHERE a.ESTAT = tj30t.ESTAT
        AND a.SPRAS = 'P'
        AND a.STSMA = tj30t.STSMA
    )
    AND tj30t.DI_OPERATION_TYPE != 'D'
)
SELECT
    aufk.AUFNR AS ordem,
    -- aufk.AUART AS tipo_ordem,
    -- aufk.BUKRS AS empresa,
    -- aufk.KTEXT AS texto_breve,
    -- aufk.ERDAT AS data_criacao,
    -- aufk.WERKS AS centro_trabalho,
    -- STRING_AGG(tj30t.TXT04, ' ') WITHIN GROUP (ORDER BY tj30t.TXT04) AS status_usuario
    -- CONCAT_WS(' ', COLLECT_LIST(tj30t.TXT04)) AS status_usuario
    -- jest.*,
    -- aufk.AUFNR as left_JEST_right_JCDS,
    jcds.*,
    aufk.AUFNR as left_JCDS_right_JSTO,
    jsto.*,
    aufk.AUFNR as left_JSTO_right_tj30,
    tj30.*,
    aufk.AUFNR as left_tj30_right_tj30t,
    tj30t.*

FROM ordens_filtradas AS aufk

LEFT JOIN jcds_filtrado AS jcds
    ON jcds.MANDT = aufk.MANDT 
    AND jcds.OBJNR = aufk.OBJNR
    AND jcds.STAT LIKE 'E%'
    AND jcds.INACT IS NULL

LEFT JOIN jsto_filtrado AS jsto
    ON jsto.OBJNR = aufk.OBJNR

LEFT JOIN tj30_filtrado AS tj30
    ON jsto.STSMA = tj30.STSMA AND jcds.STAT = tj30.ESTAT

LEFT JOIN tj30t_filtrado AS tj30t
    ON jcds.STAT = tj30t.ESTAT
    AND tj30t.SPRAS = 'P'
    AND tj30t.STSMA = jsto.STSMA

-- GROUP BY aufk.AUFNR